# Ensemble Learning

## Initial Imports

In [44]:
import warnings
warnings.filterwarnings('ignore')

In [45]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [46]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [47]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

df.describe()


,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.390633,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


## Split the Data into Training and Testing

In [48]:
# Create our features
# YOUR CODE HERE
## JR data cleanup 

X = df.drop(columns=['loan_status', 'issue_d', 'pymnt_plan', 'recoveries', 'collection_recovery_fee', 'next_pymnt_d', 'acc_now_delinq', 'num_tl_120dpd_2m', 'num_tl_30dpd'])

# Create our target
#y = # YOUR CODE HERE
# JR code

y = df["loan_status"]

## JR Note - something seemed off here - looked back to the string issue for ML > get dummies!

X_encoded = pd.get_dummies(X, columns=["home_ownership", "verification_status", "initial_list_status", "application_type"], drop_first=True)

In [49]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.390633,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [50]:
# Check the balance of our target values
# YOUR CODE HERE

y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [51]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X_encoded, y, random_state=1, stratify=y)

#JR get shape

X_train.shape

(51612, 80)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [52]:
# Create the StandardScaler instance
# YOUR CODE HERE

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [53]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE

X_scaler = scaler.fit(X_train)

In [54]:
# Scale the training and testing data
# YOUR CODE HERE

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [55]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE

# JR - sorry for the gross vomit joke;)
# JR > note to self - play w the n estimators

from imblearn.ensemble import BalancedRandomForestClassifier
barf = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
barf.fit(X_train_scaled, y_train)


In [56]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

from sklearn.metrics import balanced_accuracy_score
y_pred = barf.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)


In [57]:
# Display the confusion matrix
# YOUR CODE HERE

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

In [58]:
# Print the imbalanced classification report
# YOUR CODE HERE

from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

In [59]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE

# JR note to self - i think this is right for "descending"

imps = barf.feature_importances_
imps_sorted = sorted(zip(imps, X.columns), reverse=True)
imps_sorted[:5]

### Easy Ensemble Classifier

In [60]:
# Train the Classifier
# YOUR CODE HERE

from imblearn.ensemble import EasyEnsembleClassifier
easyclass = EasyEnsembleClassifier(n_estimators=1000, random_state=1)
easyclass = easyclass.fit(X_train_scaled, y_train)


In [61]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

from sklearn.metrics import balanced_accuracy_score
y_pred = easyclass.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

In [62]:
# Display the confusion matrix
# YOUR CODE HERE

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

In [63]:
# Print the imbalanced classification report
# YOUR CODE HERE

from imblearn.metrics import classification_report_imbalanced
y_pred = easyclass.predict(X_test_scaled)
prit(classification_report_imbalanced(y_test, y_pred))

### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

    JR > Best acc score was the early ensemble classifier

2. Which model had the best recall score?

    YOUR ANSWER HERE.

    JR > Best recall score was the balanced forest classifier

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

    JR > Best geometric mean was the early ensemble classifier

4. What are the top three features?

    YOUR ANSWER HERE.

    JR > out_prncp_inv, total_rec_prncp, total_pymnt